In [1]:
#Install the Python SDK

In [2]:
!pip install -q -U google-generativeai

In [3]:
#Import packages

In [4]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [26]:
#Setup your API key

In [6]:
GOOGLE_API_KEY=userdata.get('Mykey')

genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
#List models

In [8]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-pro
models/gemini-pro-vision


In [9]:
#**Generate text from text inputs**

In [12]:
model = genai.GenerativeModel('gemini-pro')

In [16]:
%%time
response = model.generate_content("What is life ?")

CPU times: user 91.6 ms, sys: 11 ms, total: 103 ms
Wall time: 6.72 s


In [17]:
response.text

"- **Scientific Definition:** Life is a set of complex processes that distinguish an organism from non-living matter, characterized by the ability to metabolize, reproduce, respond to stimuli, adapt to the environment, and maintain homeostasis. \n\n\n- **Philosophical Definition:**  Life is the characteristic that distinguishes a living organism from a nonliving object, involving the capacity for growth, reproduction, functional activity, and consciousness or awareness. It is the essential quality that allows an organism to exist, function, and adapt to its environment.\n\n\n- **Religious Definition:** Life is considered a sacred gift from a higher power or deity, often associated with a notion of purpose, meaning, and spiritual existence. In religious traditions, life may be viewed as a journey of self-discovery, spiritual growth, and connection with the divine.\n\n\n- **Holistic Definition:** Life is an interconnected web of relationships, processes, and phenomena that exist as a dyn

In [18]:
to_markdown(response.text)

> - **Scientific Definition:** Life is a set of complex processes that distinguish an organism from non-living matter, characterized by the ability to metabolize, reproduce, respond to stimuli, adapt to the environment, and maintain homeostasis. 
> 
> 
> - **Philosophical Definition:**  Life is the characteristic that distinguishes a living organism from a nonliving object, involving the capacity for growth, reproduction, functional activity, and consciousness or awareness. It is the essential quality that allows an organism to exist, function, and adapt to its environment.
> 
> 
> - **Religious Definition:** Life is considered a sacred gift from a higher power or deity, often associated with a notion of purpose, meaning, and spiritual existence. In religious traditions, life may be viewed as a journey of self-discovery, spiritual growth, and connection with the divine.
> 
> 
> - **Holistic Definition:** Life is an interconnected web of relationships, processes, and phenomena that exist as a dynamic and evolving system. This definition emphasizes the interdependence of all living organisms and their interactions with the environment, highlighting the complexity and interconnectedness of life's myriad manifestations.
> 
> - **Adaptive Definition:** Life is a self-organizing and adaptive process that involves the ability to respond to changes in the environment, evolve, and maintain a certain level of complexity or organization. In this sense, life exhibits the ability to adapt and adjust its structures, behaviors, and strategies to survive and thrive in diverse and changing environments.

In [19]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

In [20]:
response.candidates

[content {
  parts {
    text: "- **Scientific Definition:** Life is a set of complex processes that distinguish an organism from non-living matter, characterized by the ability to metabolize, reproduce, respond to stimuli, adapt to the environment, and maintain homeostasis. \n\n\n- **Philosophical Definition:**  Life is the characteristic that distinguishes a living organism from a nonliving object, involving the capacity for growth, reproduction, functional activity, and consciousness or awareness. It is the essential quality that allows an organism to exist, function, and adapt to its environment.\n\n\n- **Religious Definition:** Life is considered a sacred gift from a higher power or deity, often associated with a notion of purpose, meaning, and spiritual existence. In religious traditions, life may be viewed as a journey of self-discovery, spiritual growth, and connection with the divine.\n\n\n- **Holistic Definition:** Life is an interconnected web of relationships, processes, an

By default, the model returns a response after completing the entire generation process. You can also stream the response as it is being generated, and the model will return chunks of the response as soon as they are generated.

To stream responses, use GenerativeModel.generate_content(..., stream=True).

In [21]:
%%time
response = model.generate_content("What is the meaning of life?", stream=True)

CPU times: user 133 ms, sys: 18 ms, total: 151 ms
Wall time: 10.7 s


In [22]:
for chunk in response:
  print(chunk.text)
  print("_"*80)

1. **Eudaimonia (Aristotle):** Aristotle believed that the meaning
________________________________________________________________________________
 of life is eudaimonia, often translated as "flourishing" or "well-being." He argued that the highest good for humans is to live
________________________________________________________________________________
 in accordance with reason and virtue, and that this leads to a life of happiness and fulfillment.

2. **Hedonism (Epicurus):** Epicurus believed that the meaning of life is to seek pleasure and avoid pain. He argued that the highest good is to experience as much pleasure as possible, while minimizing
________________________________________________________________________________
 pain and suffering.

3. **Stoicism (Marcus Aurelius):** The Stoics believed that the meaning of life is to live in accordance with nature and reason. They argued that the highest good is to be virtuous and to accept whatever happens to us with equanimity.



When streaming, some response attributes are not available until you've iterated through all the response chunks. This is demonstrated below:


In [23]:
response = model.generate_content("What is the meaning of life?", stream=True)

The prompt_feedback attribute works:


In [24]:
response.prompt_feedback


safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

In [25]:
try:
  response.text
except Exception as e:
  print(f'{type(e).__name__}: {e}')

IncompleteIterationError: Please let the response complete iteration before accessing the final accumulated
attributes (or call `response.resolve()`)


**Chat conversations**

In [27]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

In [28]:
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
to_markdown(response.text)

> A computer is like a magic box that can do many things when you tell it what to do!

In [29]:
chat.history

[parts {
   text: "In one sentence, explain how a computer works to a young child."
 }
 role: "user",
 parts {
   text: "A computer is like a magic box that can do many things when you tell it what to do!"
 }
 role: "model"]

In [30]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?", stream=True)

for chunk in response:
  print(chunk.text)
  print("_"*80)

A computer is an electronic device that can be programmed to carry out a set of
________________________________________________________________________________
 instructions, often called a program. It consists of hardware, which includes physical components like the processor, memory, and storage, and software, which comprises the instructions
________________________________________________________________________________
 that tell the computer what to do. The processor, also known as the CPU, is the brain of the computer and executes the instructions. Memory, or RAM, temporarily stores data and instructions that are being processed, while storage devices such as hard disk drives or solid-state drives permanently store data and programs. Input devices
________________________________________________________________________________
 like keyboards and mice allow users to interact with the computer, while output devices like monitors and printers display or print the results of compu

In [31]:
for message in chat.history:
  if message.role == 'You are here yo assist a Teacher for teaching children':
    display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  else:
    display(to_markdown(f'{message.role}: {message.parts[0].text}'))

> user: In one sentence, explain how a computer works to a young child.

> model: A computer is like a magic box that can do many things when you tell it what to do!

> user: Okay, how about a more detailed explanation to a high schooler?

> model: A computer is an electronic device that can be programmed to carry out a set of instructions, often called a program. It consists of hardware, which includes physical components like the processor, memory, and storage, and software, which comprises the instructions that tell the computer what to do. The processor, also known as the CPU, is the brain of the computer and executes the instructions. Memory, or RAM, temporarily stores data and instructions that are being processed, while storage devices such as hard disk drives or solid-state drives permanently store data and programs. Input devices like keyboards and mice allow users to interact with the computer, while output devices like monitors and printers display or print the results of computations or other operations.
> 
> In summary, a computer is a programmable machine that can store, process, and output data and information under the control of a set of instructions.

**Use embeddings**

In [32]:
result = genai.embed_content(
    model="models/embedding-001",
    content="how i can teach well?",
    task_type="retrieval_document",
    title="Embedding of single string")

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED]')

[0.02260764, 0.004664133, 0.03912094, 0.0060536554 ... TRIMMED]


Note: The retrieval_document task type is the only task that accepts a title.

To handle batches of strings, pass a list of strings in content:

In [33]:
result = genai.embed_content(
    model="models/embedding-001",
    content=[
      'What is the meaning of life?',
      'How much wood would a woodchuck chuck?',
      'How does the brain work?',
      "how i can teach well?"],
    task_type="retrieval_document",
    title="Embedding of list of strings")

# A list of inputs > A list of vectors output
for v in result['embedding']:
  print(str(v)[:50], '... TRIMMED ...')

[0.0040260437, 0.004124458, -0.014209415, -0.00183 ... TRIMMED ...
[-0.004049845, -0.0075574904, -0.0073463684, -0.03 ... TRIMMED ...
[0.025310587, -0.0080734305, -0.029902633, 0.01160 ... TRIMMED ...
[0.02503743, 0.022164688, 0.041609593, 0.014568144 ... TRIMMED ...


In [34]:
response.candidates[0].content


parts {
  text: "A computer is an electronic device that can be programmed to carry out a set of instructions, often called a program. It consists of hardware, which includes physical components like the processor, memory, and storage, and software, which comprises the instructions that tell the computer what to do. The processor, also known as the CPU, is the brain of the computer and executes the instructions. Memory, or RAM, temporarily stores data and instructions that are being processed, while storage devices such as hard disk drives or solid-state drives permanently store data and programs. Input devices like keyboards and mice allow users to interact with the computer, while output devices like monitors and printers display or print the results of computations or other operations.\n\nIn summary, a computer is a programmable machine that can store, process, and output data and information under the control of a set of instructions."
}
role: "model"

In [35]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = response.candidates[0].content)

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED ...')

[0.005816124, -0.036204133, 0.004268166, 0.0242565 ... TRIMMED ...


In [36]:
chat.history

[parts {
   text: "In one sentence, explain how a computer works to a young child."
 }
 role: "user",
 parts {
   text: "A computer is like a magic box that can do many things when you tell it what to do!"
 }
 role: "model",
 parts {
   text: "Okay, how about a more detailed explanation to a high schooler?"
 }
 role: "user",
 parts {
   text: "A computer is an electronic device that can be programmed to carry out a set of instructions, often called a program. It consists of hardware, which includes physical components like the processor, memory, and storage, and software, which comprises the instructions that tell the computer what to do. The processor, also known as the CPU, is the brain of the computer and executes the instructions. Memory, or RAM, temporarily stores data and instructions that are being processed, while storage devices such as hard disk drives or solid-state drives permanently store data and programs. Input devices like keyboards and mice allow users to interact with

In [37]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = chat.history)

# 1 input > 1 vector output
for i,v in enumerate(result['embedding']):
  print(str(v)[:50], '... TRIMMED...')

[-0.014632266, -0.042202696, -0.015757175, 0.01548 ... TRIMMED...
[-0.03704981, -0.02199058, 0.011430039, 0.01014303 ... TRIMMED...
[-0.010055617, -0.07208932, -0.00011750793, -0.023 ... TRIMMED...
[0.005816124, -0.036204133, 0.004268166, 0.0242565 ... TRIMMED...


In [38]:
#response = model.generate_content('[Questionable prompt here]')
response = model.generate_content('[babies are disturbing my lunch time . what should I do ?]')
response.candidates

[content {
  parts {
    text: "1. **Communicate with the Parents**:\n   - Respectfully ask the parents of the babies if they could find a quieter area for the babies to play or feed so that you can enjoy your lunch break in peace.\n   - Express your concerns to them in a friendly and understanding manner.\n\n2. **Adjust Your Lunch Schedule**:\n   - If possible, try to adjust your lunch break to a time when the babies are likely to be napping or engaged in other activities that are less disruptive.\n\n3. **Find a Quieter Spot**:\n   - Look for an alternative, quieter spot to eat your lunch where you are less likely to be disturbed by the babies. This could be a break room, a conference room, or even your car.\n\n4. **Use Noise-Canceling Headphones**:\n   - Consider wearing noise-canceling headphones while eating your lunch to minimize the sound of the babies\' cries or other noises.\n\n5. **Ask for Flexibility in Work Hours**:\n   - If the situation with the babies is causing significa

The prompt_feedback will tell you which safety filter blocked the prompt:


In [39]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

**bold Now provide the same prompt to the model with newly configured safety settings, and you may get a response.text**

In [40]:
#response = model.generate_content('[Questionable prompt here]',
                                  #safety_settings={'HARASSMENT':'block_none'})
response = model.generate_content('[babies are disturbing my lunch time . what should I do ?]',
                                  safety_settings={'HARASSMENT':'block_none'})
response.text


"1. **Communicate with the Parents**:\n   - Politely approach the parents of the babies and express your concerns. Explain that their babies' crying is affecting your lunch break.\n   - Suggest that they take their babies to a quieter area to soothe or feed them.\n\n2. **Request Quiet Zone**:\n   - Inquire if your workplace or building has a designated quiet zone or break area where you can go to have your lunch in peace.\n   - If there is no such area, consider creating one to provide a distraction-free space for employees.\n\n3. **Noise-Canceling Headphones**:\n   - Invest in a good pair of noise-canceling headphones. These headphones will effectively block out most of the background noise, allowing you to focus on your meal.\n\n4. **Ear Plugs**:\n   - Ear plugs are an affordable and portable alternative to noise-canceling headphones. They can be used to reduce the volume of the babies' crying.\n\n5. **Ask for Help**:\n   - If the parents are unresponsive or unable to control their b

## New Section

###Encode messages
